# Credits

**a)** All Jared

**b)** Mostly Maria, everyone brainstormed priors in class.

**c)** All Maria

In [ ]:
import numpy as np
import pandas as pd

import arviz as az

import bebi103

# Import Altair for high level plotting
import altair as alt
import altair_catplot as altcat
# Pevent bulky altair plots
alt.data_transformers.enable('json')


import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

## Problem 8.2: Microtubule catastrophe, 40 pts

_Note: This problem is best done after the lecture November 22._

In this problem, we use data from [Gardner, Zanic, et al., Depolymerizing kinesins Kip3 and MCAK shape cellular microtubule architecture by differential control of catastrophe, *Cell*, **147**, 1092-1103, 2011](https://doi.org/10.1016/j.cell.2011.10.037). The authors investigated the dynamics of microtubule catastrophe, the switching of a microtubule from a growing to a shrinking state.  In particular, they were interested in the time between the start of growth of a microtubule and the catastrophe event. They monitored microtubules in a single-molecule [TIRF assay](https://en.wikipedia.org/wiki/Total_internal_reflection_fluorescence_microscope) by using tubulin (the monomer that comprises a microtubule) that was labeled with a fluorescent marker. As a control to make sure that fluorescent labels and exposure to laser light did not affect the microtubule dynamics, they performed a similar experiment using differential interference contrast (DIC) microscopy. They measured the time until catastrophe with labeled and unlabeled tubulin. We will carefully analyze the data and make some conclusions about the processes underlying microtubule catastrophe.

In the file `gardner_mt_catastrophe_only_tubulin.csv` (which you can download [here](../data/gardner_mt_catastrophe_only_tubulin.csv)), we have observed catastrophe times of microtubules with different concentrations of tubulin. To start with, we will consider the experiment run with a tubulin concentration of 12 µM. So, our data set consists of a set of measurements of the amount of time to catastrophe. We will consider three models for microtubule catastrophe.

- Model 1: The time to catastrophe is Exponentially distributed.
- Model 2: The time to catastrophe is Gamma distributed.
- Model 3: The time to catastrophe is Weibull distributed.

Note that these descriptions are for the likelihood; we have not specified priors.


**a)  Describe the three models in words. Give physical descriptions of the meanings of their parameters. Describe how these models are related to each other. Tutorial 3c will be useful.** 


- Model 1: The time to catastrophe is Exponentially distributed.

This suggests that the occurance of catastrophe is a Poisson process, so it is a "rare event" that requires multiple subprocesses to lead it it. The parameter for the process, if it is exponential, $\beta$, represents the characteristic rate of catastrophe, that is how often catastrophe happens in a certain amount of time. It can also be parametrized as $\tau=1/\beta$, the characteristic catastrophe time, which fits what we are given in our data. The Exponential distribution is a special case of the Gamma distribution where $\alpha = 1$ and a special case of the Weibull distribution where $\alpha = 1$ and $\sigma=1/\beta$



- Model 2: The time to catastrophe is Gamma distributed.

This suggests that the occurance of catastrophe represents a specific number of occurances of a Poisson process, that is a discrete number of steps that occur at the same rate must occur for catastrophe to occur. There are two parameters for this distribution, $\alpha$ and $\beta$, where $\alpha$ is the number of arrivals (or "steps") required to trigger catastrophe, and $\beta$ is the rate of the arrivals. Thus, the characteristic catastrophe time is given by $\alpha/\beta$. 


- Model 3: The time to catastrophe is Weibull distributed.

This suggests that the likelihood of catastrophe is dependent on the amount of time it has been since the last catastrophe, so the longer it has been since the last catastrophe, the more likely it is that catastrophe will occur. There are two parameters for this distribution, $\alpha$ which defines how the probability changes over time, and $\sigma$ which is the characteristic catastrophe time.

**b) Perform parameter estimates for the respective models and make model comparisons. Comment on what this means with respect to our understanding of how microtubule catastrophe works.**

We first load in our data and come up with our priors for the parameters of the three distributions. We don't have much prior knowledge so we will keep them simple (normal distribution).

**Exponential(beta)**

tau ~ normal(400, 100)

beta = 1/tau

We guess that the castastrophe times will be in the hundreds.

**Gamma($\alpha$, $\beta$)**

alpha ~ normal(10, 3)

We assume that it will take around 10 arrivals to trigger catastrophe.

tau ~ normal(150, 50)

beta = 1/tau


**Weibull($\alpha$, $\sigma$)**

$\alpha$ ~ normal(1, 0.1)

We think that the rate of catastrophe is not dependent on the time elapsed from the last event, so we center our distribution aroud 1 without much variation.

$\sigma$ ~ normal(10, 3)


In [3]:
df = pd.read_csv('../data/gardner_mt_catastrophe_only_tubulin.csv', comment = "#")

In [4]:
# Check out the data we are working with
df.head()

,12 uM,7 uM,9 uM,10 uM,14 uM
0,25.000,35.0,25.0,50.0,60.0
1,40.000,45.0,40.0,60.0,75.0
2,40.000,50.0,40.0,60.0,75.0
3,45.429,50.0,45.0,75.0,85.0
4,50.000,55.0,50.0,75.0,115.0


In [5]:
prior_sm1 = bebi103.stan.StanModel(file='./8.2_prior_pred_12_m1.stan')
prior_sm2 = bebi103.stan.StanModel(file='./8.2_prior_pred_12_m2.stan')
prior_sm3 = bebi103.stan.StanModel(file='./8.2_prior_pred_12_m3.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_c0903ec82a19de1bfb9fb132714a9c40 NOW.


Using cached StanModel.


/Users/jaredlivingston/anaconda3/lib/python3.6/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /var/folders/w3/tn6brvj12bx95d_v766ht54h0000gn/T/tmpgv_ymvzi/stanfit4anon_model_c0903ec82a19de1bfb9fb132714a9c40_3422403499193077404.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_43e7521b063a968e193cf3a1be7574a6 NOW.
/Users/jaredlivingston/anaconda3/lib/python3.6/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /var/folders/w3/tn6brvj12bx95d_v766ht54h0000gn/T/tmp3vt1b5pr/stanfit4anon_model_43e7521b063a968e193cf3a1be7574a6_8102683413684293024.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [ ]:
# Store input parameters in a dictionary so stan can access them
data = dict(N=692)

# Generate samples
samples_gen1 = prior_sm1.sampling(data=data,
                          algorithm='Fixed_param',
                          warmup=0,
                          chains=1,
                          iter=1000)
samples_gen2 = prior_sm2.sampling(data=data,
                          algorithm='Fixed_param',
                          warmup=0,
                          chains=1,
                          iter=1000)
samples_gen3 = prior_sm3.sampling(data=data,
                          algorithm='Fixed_param',
                          warmup=0,
                          chains=1,
                          iter=1000)

# Store samples in a dataframe
df_gen1 = bebi103.stan.to_dataframe(samples_gen1, diagnostics=False)
df_gen2 = bebi103.stan.to_dataframe(samples_gen2, diagnostics=False)
df_gen3 = bebi103.stan.to_dataframe(samples_gen3, diagnostics=False)

# Let's look at one of the dataframes to make sure they look ok
df_gen3.head()

Yay! It looks ok so we can move on to plotting our prior predictive checks.

In [ ]:
p = bebi103.viz.predictive_ecdf(samples_gen1, "time",
                                x_axis_label = "intercatastrophe time (s)")
p.x_range = bokeh.models.Range1d(-10, 6000)
bokeh.io.show(p)

In [ ]:
p = bebi103.viz.predictive_ecdf(samples_gen2, "time",
                                x_axis_label = "intercatastrophe time (s)")
p.x_range = bokeh.models.Range1d(-10, 3000)
bokeh.io.show(p)

In [ ]:
p = bebi103.viz.predictive_ecdf(samples_gen3, "time",
                                x_axis_label = "intercatastrophe time (s)")
p.x_range = bokeh.models.Range1d(-10, 1000)
bokeh.io.show(p)

Our priors look reasonable, since times are centered in the hundreds, so now we can move on to creating our mcmc models and sampling.

In [ ]:
sm1 = bebi103.stan.StanModel(file='./8.2_mcmc_12_m1.stan')
sm2 = bebi103.stan.StanModel(file='./8.2_mcmc_12_m2.stan')
sm3 = bebi103.stan.StanModel(file='./8.2_mcmc_12_m3.stan')

In [ ]:
data = dict(N=len(df),
           time=df['12 uM'].values.astype(float))

In [ ]:
samples1 = sm1.sampling(data=data)
samples2 = sm2.sampling(data=data)
samples3 = sm3.sampling(data=data)

In [ ]:
df_mcmc1 = bebi103.stan.to_dataframe(samples1, diagnostics=False, inc_warmup=False)
df_mcmc2 = bebi103.stan.to_dataframe(samples2, diagnostics=False, inc_warmup=False)
df_mcmc3 = bebi103.stan.to_dataframe(samples3, diagnostics=False, inc_warmup=False)

We now plot the post predictive checks to do model comparison.

In [ ]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples1, 
                                          percentiles=[99, 70, 50, 30],
                                          name='time_ppc', 
                                          data=df['12 uM'].values,
                                          diff=True,
                                          data_line=False))

We can see there is a systematic issue with using the exponential distribution becuase the two inflection points will always be in the wrong place.

In [ ]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples2, 
                                          percentiles=[99, 70, 50, 30],
                                          name='time_ppc', 
                                          data=df['12 uM'].values,
                                          diff=True,
                                          data_line=False))

The gamma distribution looks a lot more promising! Most of our values fall into that innermost confidence interval.

In [ ]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples3, 
                                          percentiles=[99, 70, 50, 30],
                                          name='time_ppc', 
                                          data=df['12 uM'].values,
                                          diff=True,
                                          data_line=False))

Ooof the weibull distribution model is also not great (better than exponential though). Only a few of our datapoints will fall in that 99% confidence interval, so the curve isn't a very good match.

To confirm quantitatively, we compute the loo given the log likelihood and use the bebi103 compare function to calculate the loo and the weights.

In [ ]:
bebi103.stan.compare({'exponential': samples1, 'gamma': samples2, 'weibull': samples3},
                     log_likelihood='log_like', ic='loo')

We can see that gamma (Model 2) is the best distribution to fit our data because the weight is much higher (0.95 vs 10^-2 and 10^-13). Based on the visual analysis of the post predictive checks above,  this makes a lot of sense! 

**c) Using whichever model you favor based on your work in part (b), obtain parameter estimates for the other tubulin concentrations. Given that microtubules polymerize faster with higher tubulin concentrations, is there anything you can say about the occurrence of catastrophe by looking at the values of the parameters versus tubulin concentration?**


We picked the gamma distribution model so we will now sample out of it for all 5 concentrations in the dataframe.

In [ ]:
data12 = dict(N=len(df),
           time=df['12 uM'].values.astype(float))
data7 = dict(N=len(df['7 uM'].dropna()),
           time=df['7 uM'].dropna().values.astype(float))
data9 = dict(N=len(df['9 uM'].dropna()),
           time=df['9 uM'].dropna().values.astype(float))
data10 = dict(N=len(df['10 uM'].dropna()),
           time=df['10 uM'].dropna().values.astype(float))
data14 = dict(N=len(df['14 uM'].dropna()),
           time=df['14 uM'].dropna().values.astype(float))

In [ ]:
samples12 = sm2.sampling(data=data12)
samples7 = sm2.sampling(data=data7)
samples9 = sm2.sampling(data=data9)
samples10 = sm2.sampling(data=data10)
samples14 = sm2.sampling(data=data14)

We now preform post predictive checks and can conlcude that the gamma model is resoanble for all 5 concentrations.

In [ ]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples12, 
                                          percentiles=[99, 70, 50, 30],
                                          name='time_ppc', 
                                          data=df['12 uM'].values,
                                          diff=True,
                                          data_line=False,
                                          title = "12 uM",
                                          x_axis_label = "intercatastrophe time (s)"))

In [ ]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples7, 
                                          percentiles=[99, 70, 50, 30],
                                          name='time_ppc', 
                                          data=df['7 uM'].dropna().values,
                                          diff=True,
                                          data_line=False,
                                          title = "7 uM",
                                          x_axis_label = "intercatastrophe time (s)"))

In [ ]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples9, 
                                          percentiles=[99, 70, 50, 30],
                                          name='time_ppc', 
                                          data=df['9 uM'].dropna().values,
                                          diff=True,
                                          data_line=False,
                                          title = "9 uM",
                                          x_axis_label = "intercatastrophe time (s)"))

In [ ]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples10, 
                                          percentiles=[99, 70, 50, 30],
                                          name='time_ppc', 
                                          data=df['10 uM'].dropna().values,
                                          diff=True,
                                          data_line=False,
                                          title = "10 uM",
                                          x_axis_label = "intercatastrophe time (s)"))

In [ ]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples14, 
                                          percentiles=[99, 70, 50, 30],
                                          name='time_ppc', 
                                          data=df['14 uM'].dropna().values,
                                          diff=True,
                                          data_line=False,
                                          title = "14 uM",
                                          x_axis_label = "intercatastrophe time (s)"))

Now we want to examine the occurrence of catastrophe by looking at the values of the parameters versus tubulin concentration?

There are two parameters for this distribution,  𝛼  and  𝛽 , where  𝛼  is the number of arrivals (or "steps") required to trigger catastrophe, and  𝛽  is the rate of the arrivals. Thus, the characteristic catastrophe time is given by  𝛼/𝛽 .

We first need to put our sampled values in a dataframe so we can look at the parameters

In [ ]:
df_mcmc7 = bebi103.stan.to_dataframe(samples7, diagnostics=False, inc_warmup=False)
df_mcmc9 = bebi103.stan.to_dataframe(samples9, diagnostics=False, inc_warmup=False)
df_mcmc10 = bebi103.stan.to_dataframe(samples10, diagnostics=False, inc_warmup=False)
df_mcmc12 = bebi103.stan.to_dataframe(samples12, diagnostics=False, inc_warmup=False)
df_mcmc14 = bebi103.stan.to_dataframe(samples14, diagnostics=False, inc_warmup=False)

Let's make a column labeling each dataframe for ease of plotting:

In [ ]:
df_mcmc7["Concentration"] = 7
df_mcmc9["Concentration"] = 9
df_mcmc10["Concentration"] = 10
df_mcmc12["Concentration"] = 12
df_mcmc14["Concentration"] = 14

Now let's take a look at the 𝛼 values for each concentration. This will give us an idea of the amount of steps required to trigger catastrophe:

In [ ]:
# First let's make a helper plotting function
def make_ecdf(dataframe, data, title):
    c = altcat.catplot(data=dataframe,
               mark='line',
               encoding=dict(x=alt.X(data,
                                     scale=alt.Scale(
                                           clamp=True)),
                            color = alt.Color("Concentration")),
               transform='ecdf'
              ).properties(height=300,
                           width=300,
                           title = title)
    return c

# Now let's plot
(make_ecdf(df_mcmc7, "alpha_:Q", "All Experiments") 
+ make_ecdf(df_mcmc9, "alpha_:Q", "All Experiments") 
+ make_ecdf(df_mcmc10, "alpha_:Q", "All Experiments")
+ make_ecdf(df_mcmc12, "alpha_:Q", "All Experiments") 
+ make_ecdf(df_mcmc14, "alpha_:Q", "All Experiments"))



In general 𝛼 appears to increase with increasing concentration, although this is not true for concentration 12. This indicates that with higher concentration, it takes more steps to trigger catastrophe. Let's print the average alpha for each concentration for good measure:

In [ ]:
a_7 = df_mcmc7["alpha_"].mean() 
a_9 = df_mcmc9["alpha_"].mean() 
a_10 = df_mcmc10["alpha_"].mean()
a_12 = df_mcmc12["alpha_"].mean() 
a_14 = df_mcmc14["alpha_"].mean() 

{"7" : a_7, "9" : a_9, "10" : a_10, "12" : a_12, "14" : a_14}

Now let's take a look at $\beta$

In [ ]:
(make_ecdf(df_mcmc7, "beta_:Q", "All Experiments") 
+ make_ecdf(df_mcmc9, "beta_:Q", "All Experiments") 
+ make_ecdf(df_mcmc10, "beta_:Q", "All Experiments")
+ make_ecdf(df_mcmc12, "beta_:Q", "All Experiments") 
+ make_ecdf(df_mcmc14, "beta_:Q", "All Experiments"))

In [ ]:
b_7 = df_mcmc7["beta_"].mean() 
b_9 = df_mcmc9["beta_"].mean() 
b_10 = df_mcmc10["beta_"].mean() 
b_12 = df_mcmc12["beta_"].mean() 
b_14 = df_mcmc14["beta_"].mean() 

{"7" : b_7, "9" : b_9, "10" : b_10, "12" : b_12, "14" : b_14}

We can also calculate the characteristic catastrophe time (given by  𝛼/𝛽 ) for each concentration.

In [ ]:
df_mcmc7["alpha/beta"] = df_mcmc7["alpha_"] / df_mcmc7["beta_"]
df_mcmc9["alpha/beta"] = df_mcmc9["alpha_"] / df_mcmc9["beta_"]
df_mcmc10["alpha/beta"] = df_mcmc10["alpha_"] / df_mcmc10["beta_"]
df_mcmc12["alpha/beta"] = df_mcmc12["alpha_"] / df_mcmc12["beta_"]
df_mcmc14["alpha/beta"] = df_mcmc14["alpha_"] / df_mcmc14["beta_"]

Let's take a look.

In [ ]:
(make_ecdf(df_mcmc7, "alpha/beta:Q", "All Experiments") 
+ make_ecdf(df_mcmc9, "alpha/beta:Q", "All Experiments") 
+ make_ecdf(df_mcmc10, "alpha/beta:Q", "All Experiments")
+ make_ecdf(df_mcmc12, "alpha/beta:Q", "All Experiments") 
+ make_ecdf(df_mcmc14, "alpha/beta:Q", "All Experiments"))

As concentration increases, the characteristic catastrophe time seems to generally increase (the 9uM data breaks this trend). This makes sense given that 

In [ ]:
ab_7 = df_mcmc7["alpha/beta"].mean() 
ab_9 = df_mcmc9["alpha/beta"].mean() 
ab_10 = df_mcmc10["alpha/beta"].mean() 
ab_12 = df_mcmc12["alpha/beta"].mean() 
ab_14 = df_mcmc14["alpha/beta"].mean() 

{"7" : ab_7, "9" : ab_9, "10" : ab_10, "12" : ab_12, "14" : ab_14}

Finally, I will print out all the model / prior predictive check code for your viewing pleasure.

In [ ]:
print("-------------MODEL CHECK 1-------------")
print(prior_sm1.model_code)
print("-------------MODEL CHECK 2-------------")
print(prior_sm2.model_code)
print("-------------MODEL CHECK 3-------------")
print(prior_sm3.model_code)

In [ ]:
print("-------------MODEL 1-------------")
print(sm1.model_code)
print("-------------MODEL 2-------------")
print(sm2.model_code)
print("-------------MODEL 3-------------")
print(sm3.model_code)